In [1]:
from teacher.datasets import load_adult, load_beer, load_german, load_compas, load_heloc, load_pima, load_breast
from teacher.neighbors import LoreNeighborhood, NotFittedError
from teacher.metrics import coverage, precision
from teacher.explanation import FDTExplainer
from teacher.fuzzy import get_fuzzy_variables, get_fuzzy_points, dataset_membership, FuzzyContinuousSet
from teacher.tree import Rule, FDT
from teacher.explanation import mr_factual, m_factual, c_factual, f_counterfactual, i_counterfactual
from teacher.explanation._factual import _robust_threshold, _fired_rules, _get_class_fired_rules


from teacher.tree.tests.fdt_legacy_tree import FDT_Legacy

import random
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets


import numpy as np
import pandas as pd
from functools import reduce


In [2]:
def _get_fuzzy_element(fuzzy_X, idx):
    element = {}
    for feat in fuzzy_X:
        element[feat] = {}
        for fuzzy_set in fuzzy_X[feat]:
            try:
                element[feat][str(fuzzy_set)] = pd.to_numeric(fuzzy_X[feat][fuzzy_set][idx])
            except ValueError:
                element[feat][str(fuzzy_set)] = fuzzy_X[feat][fuzzy_set][idx]

    return element

In [3]:
dataset = load_beer()

df = dataset['df']
class_name = dataset['class_name']

X = df.drop(class_name, axis=1)
y = df[class_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

df_categorical_columns = dataset['discrete']
class_name = dataset['class_name']
df_categorical_columns.remove(class_name)

X_num = X[dataset['continuous']]
num_cols = X_num.columns
fuzzy_points = get_fuzzy_points('entropy', num_cols, X_num, y)

discrete_fuzzy_values = {col: df[col].unique() for col in df_categorical_columns}
fuzzy_variables_order = {col: i for i, col in enumerate(X.columns)}
fuzzy_variables = get_fuzzy_variables(fuzzy_points, discrete_fuzzy_values, fuzzy_variables_order)

df_train_membership = dataset_membership(X_train, fuzzy_variables)
df_test_membership = dataset_membership(X_test, fuzzy_variables)

print(f'Training FDT')
fdt = FDT(fuzzy_variables)
fdt.fit(X_train, y_train)

rules = fdt.to_rule_based_system()

Training FDT


In [4]:
rules_tested = len(X_test)
for fuzzy_element_idx in range(rules_tested):
    instance = X_test.iloc[fuzzy_element_idx].to_numpy().reshape(1, -1)
    fuzzy_element = _get_fuzzy_element(df_test_membership, fuzzy_element_idx)
    target = fdt.predict(instance)
    m_fact = m_factual(fuzzy_element, rules, target)
    mr_fact = mr_factual(fuzzy_element, rules, target)
    c_fact = c_factual(fuzzy_element, rules, target, lam=0.1)
    if len(mr_fact) > 1:
        print(fuzzy_element_idx)

1
21
31
77
79


In [5]:
def check_robustness(instance, r_th, factual):
    fact_AD = 0
    for rule in factual:
        fact_AD += rule.matching(instance) * rule.weight
    
    return fact_AD >= r_th

In [6]:
def factual_stats(fuzzy_element, target, rules):
    m_fact = m_factual(fuzzy_element, rules, target)
    mr_fact = mr_factual(fuzzy_element, rules, target)
    c_fact = c_factual(fuzzy_element, rules, target, lam=0.1)
    fired_rules = _fired_rules(fuzzy_element, rules)
    class_fired_rules = _get_class_fired_rules(fired_rules, target)
    class_fired_rules.sort(key=lambda rule: rule.matching(fuzzy_element) * rule.weight, reverse=True)
    beta_1 = reduce(lambda x, y: x + (y.matching(fuzzy_element) * y.weight), class_fired_rules, 0) / 2
    r_th = _robust_threshold(fuzzy_element, rules, target)
    c_beta1_fact = c_factual(fuzzy_element, rules, target, lam=0.1, beta=beta_1)
    print(f'Fired rules: {len(fired_rules)}')
    print(f'M_fact: length={len(m_fact)}, robust={check_robustness(fuzzy_element, r_th, m_fact)}')
    print(f'MR_fact: length={len(mr_fact)}, robust={check_robustness(fuzzy_element, r_th, mr_fact)}')
    print(f'C_fact: length={len(c_fact)}, robust={check_robustness(fuzzy_element, r_th, c_fact)}')
    print(f'C_beta1_fact: length={len(c_beta1_fact)}, robust={check_robustness(fuzzy_element, r_th, c_beta1_fact)}')


In [7]:
fuzzy_element_idx = 21
instance = X_test.iloc[fuzzy_element_idx].to_numpy().reshape(1, -1)
fuzzy_element = _get_fuzzy_element(df_test_membership, fuzzy_element_idx)
target = fdt.predict(instance)
factual_stats(fuzzy_element, target, rules)

Fired rules: 10
M_fact: length=3, robust=True
MR_fact: length=3, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=2, robust=False


In [8]:
rules_tested = len(X_train)
for fuzzy_element_idx in range(rules_tested):
    instance = X_train.iloc[fuzzy_element_idx].to_numpy().reshape(1, -1)
    fuzzy_element = _get_fuzzy_element(df_train_membership, fuzzy_element_idx)
    target = fdt.predict(instance)
    m_fact = m_factual(fuzzy_element, rules, target)
    mr_fact = mr_factual(fuzzy_element, rules, target)
    c_fact = c_factual(fuzzy_element, rules, target, lam=0.1)
    if len(mr_fact) > 1:
        print(f'Element {fuzzy_element_idx}')
        factual_stats(fuzzy_element, target, rules)
        print('-------------------')

Element 14
Fired rules: 8
M_fact: length=2, robust=False
MR_fact: length=3, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=2, robust=False
-------------------
Element 23
Fired rules: 10
M_fact: length=3, robust=True
MR_fact: length=3, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=2, robust=False
-------------------
Element 25
Fired rules: 4
M_fact: length=1, robust=False
MR_fact: length=2, robust=True
C_fact: length=2, robust=True
C_beta1_fact: length=2, robust=True
-------------------
Element 36
Fired rules: 10
M_fact: length=2, robust=True
MR_fact: length=2, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=2, robust=True
-------------------
Element 50
Fired rules: 8
M_fact: length=1, robust=False
MR_fact: length=2, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=1, robust=False
-------------------
Element 56
Fired rules: 6
M_fact: length=1, robust=False
MR_fact: length=2, robust=True
C_fact: length=1, robust=False
C_be

In [9]:
fuzzy_element_idx = 14

instance = X_train.iloc[fuzzy_element_idx].to_numpy().reshape(1, -1)
fuzzy_element = _get_fuzzy_element(df_train_membership, fuzzy_element_idx)
target = fdt.predict(instance)
m_fact = m_factual(fuzzy_element, rules, target)
mr_fact = mr_factual(fuzzy_element, rules, target)
c_fact = c_factual(fuzzy_element, rules, target, lam=0.1)
fired_rules = _fired_rules(fuzzy_element, rules)
class_fired_rules = _get_class_fired_rules(fired_rules, target)
class_fired_rules.sort(key=lambda rule: rule.matching(fuzzy_element) * rule.weight, reverse=True)
beta_1 = reduce(lambda x, y: x + (y.matching(fuzzy_element) * y.weight), class_fired_rules, 0) / 2
r_th = _robust_threshold(fuzzy_element, rules, target)
c_beta1_fact = c_factual(fuzzy_element, rules, target, lam=0.1, beta=beta_1)
c_beta2_fact = c_factual(fuzzy_element, rules, target, lam=0.1, beta=r_th)
factual_stats(fuzzy_element, target, rules)
print(mr_fact)
fired_rules = _fired_rules(fuzzy_element, rules)


Fired rules: 8
M_fact: length=2, robust=False
MR_fact: length=3, robust=True
C_fact: length=1, robust=False
C_beta1_fact: length=2, robust=False
[Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515), Rule((('color', '16.0'), ('strength', '0.077')), IPA, 0.6518218623481782), Rule((('color', '12.0'), ('strength', '0.092')), IPA, 0.43677042801556426)]


In [12]:
m_fact

[Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515),
 Rule((('color', '16.0'), ('strength', '0.077')), IPA, 0.6518218623481782)]

In [13]:
c_fact

[Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515)]

In [101]:
c_beta2_fact

[Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515),
 Rule((('color', '16.0'), ('strength', '0.077')), IPA, 0.6518218623481782),
 Rule((('color', '12.0'), ('strength', '0.092')), IPA, 0.43677042801556426)]

In [45]:
target

array([['IPA']], dtype='<U3')

In [35]:
def apply_rule(rule, instance, fuzzy_vars):
    changes = {a[0]: a[1] for a in rule.antecedent}
    n_inst = instance.copy()
    n_changes = 0
    changes_made = []
    for i, fv in enumerate(fuzzy_vars):
        fs_idx = np.argmax([a[0] for a in fv.membership(instance[:,i]).values()])
        if fv.name in changes and changes[fv.name] != fv.fuzzy_sets[fs_idx].name:
            n_changes += 1
            changes_made.append((fv.name, changes[fv.name]))
            if isinstance(fv.fuzzy_sets[fs_idx], FuzzyContinuousSet):
                n_inst[:,i] = float(changes[fv.name])
            else:
                n_inst[:,i] = changes[fv.name]

    return n_inst, n_changes, changes_made

In [126]:
def cf_stats(cf, inst, target, fuzzy_vars, fdt):
    for i, (r, d) in enumerate(cf):
        n_instance, n_changes, changes_made = apply_rule(r, inst, fuzzy_vars)
        n_target = fdt.predict(n_instance)
        print(changes_made)
        if n_target != target: 
            break
    # Number of changes, rules tested, distance
    return changes_made, n_changes, i + 1, d

In [40]:
i_cf = i_counterfactual(fuzzy_element, rules, target, dataset['continuous'])
print(cf_stats(i_cf, instance, target, fuzzy_variables, fdt))

([('color', '16.0')], 1, 2, 0.05555555555555555)


In [104]:
i_cf[0]

(Rule((('color', '12.0'), ('strength', '0.092')), Barleywine, 0.5632295719844357),
 0.0)

In [105]:
a = [r for r in i_cf if r[1] > 0][:4]
a

[(Rule((('color', '16.0'), ('strength', '0.092')), Barleywine, 0.8792032717358376),
  0.05555555555555555),
 (Rule((('color', '12.0'), ('strength', '0.077')), Barleywine, 0.07361222847948515),
  0.10666666666666678),
 (Rule((('color', '7.0'), ('bitterness', '60.0'), ('strength', '0.092')), Barleywine, 0.03981776765375854),
  0.1111111111111111),
 (Rule((('color', '16.0'), ('strength', '0.077')), Barleywine, 0.348178137651822),
  0.16222222222222232)]

In [127]:
f_cf = f_counterfactual(mr_fact, fuzzy_element, rules, target, dataset['continuous'], tau=0.5)
print(cf_stats(f_cf, instance, target, fuzzy_variables, fdt))

[('strength', '0.077')]
[('color', '16.0'), ('strength', '0.077')]
[]
[('color', '16.0')]
([('color', '16.0')], 1, 4, 0.1470370370370369)


In [42]:
fuzzy_points

{'color': [0.0, 4.0, 5.0, 7.0, 12.0, 16.0, 19.0, 20.0, 29.0, 45.0],
 'bitterness': [8.0, 19.0, 26.0, 33.0, 36.0, 60.0, 250.0],
 'strength': [0.039, 0.055, 0.068, 0.077, 0.092, 0.136]}

In [57]:
fired_rules

[Rule((('color', '12.0'), ('strength', '0.077')), Barleywine, 0.07361222847948515),
 Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515),
 Rule((('color', '12.0'), ('strength', '0.092')), Barleywine, 0.5632295719844357),
 Rule((('color', '12.0'), ('strength', '0.092')), IPA, 0.43677042801556426),
 Rule((('color', '16.0'), ('strength', '0.077')), Barleywine, 0.348178137651822),
 Rule((('color', '16.0'), ('strength', '0.077')), IPA, 0.6518218623481782),
 Rule((('color', '16.0'), ('strength', '0.092')), Barleywine, 0.8792032717358376),
 Rule((('color', '16.0'), ('strength', '0.092')), IPA, 0.1207967282641624)]

In [43]:
y.unique()

array(['Blanche', 'Lager', 'Pilsner', 'IPA', 'Stout', 'Barleywine',
       'Porter', 'Belgian-Strong-Ale'], dtype=object)

In [80]:
fuzzy_points['bitterness']

[8.0, 19.0, 26.0, 33.0, 36.0, 60.0, 250.0]

In [83]:
VARIABLES_DICT = {
    'color':{
        '5.0': 'extremely \ low',
        '7.0': 'very \ low',
        '12.0': 'low',
        '16.0': 'high',
        '19.0': 'very \ high',
        '20.0': 'extremely \ high',
    },
    'strength': {
        '0.039': 'extremely \ low',
        '0.055': 'very \ low',
        '0.068': 'low',
        '0.077':'high',
        '0.092': 'very \ high',
        '0.136': 'extremely \ high',
    },
    'bitterness': {
        '8.0':  'extremely \ low',
        '19.0':  'very \ low',
        '26.0':  'low',
        '33.0':  'medium',
        '36.0':  'high',
        '60.0':  'very \ high',
        '250.0':'extremely \ high',
    }
}

In [84]:
def latex_rule(rule, instance, idx):
    w = rule.weight
    ad = w * rule.matching(instance)
    lr = ""
    lr += "\\tcr{ \(\n"
    lr += "\small\n"
    lr += "\\begin{array}{ll}\n"
    lr += f"r_{idx}: & ({rule.antecedent[0][0]} \ {VARIABLES_DICT[rule.antecedent[0][0]][rule.antecedent[0][1]]}) \land ({rule.antecedent[1][0]} \ {VARIABLES_DICT[rule.antecedent[1][0]][rule.antecedent[1][1]]})  \\\\ \n"
    lr += f"&w(r_{idx}) = {w:.2f}, \ AD(r_{idx}(x),x) = {ad:.2f} \n"
    lr += "\end{array}\n"
    lr += "\)\n"
    lr += "\smallskip\n"
    lr += "}\n"
    return lr

In [86]:
fired_rules

[Rule((('color', '12.0'), ('strength', '0.077')), Barleywine, 0.07361222847948515),
 Rule((('color', '12.0'), ('strength', '0.077')), IPA, 0.926387771520515),
 Rule((('color', '12.0'), ('strength', '0.092')), Barleywine, 0.5632295719844357),
 Rule((('color', '12.0'), ('strength', '0.092')), IPA, 0.43677042801556426),
 Rule((('color', '16.0'), ('strength', '0.077')), Barleywine, 0.348178137651822),
 Rule((('color', '16.0'), ('strength', '0.077')), IPA, 0.6518218623481782),
 Rule((('color', '16.0'), ('strength', '0.092')), Barleywine, 0.8792032717358376),
 Rule((('color', '16.0'), ('strength', '0.092')), IPA, 0.1207967282641624)]

In [94]:
idx = 1
for cv in y.unique():
    print(f"\item \\tcr Class ${cv}$")
    rs = [r for r in fired_rules if r.consequent == cv]
    for r in rs:
        print(latex_rule(r, fuzzy_element, idx))
        idx += 1

\item \tcr Class $Blanche$
\item \tcr Class $Lager$
\item \tcr Class $Pilsner$
\item \tcr Class $IPA$
\tcr{ \(
\small
\begin{array}{ll}
r_1: & (color \ low) \land (strength \ high)  \\ 
&w(r_1) = 0.93, \ AD(r_1(x),x) = 0.43 
\end{array}
\)
\smallskip
}

\tcr{ \(
\small
\begin{array}{ll}
r_2: & (color \ low) \land (strength \ very \ high)  \\ 
&w(r_2) = 0.44, \ AD(r_2(x),x) = 0.22 
\end{array}
\)
\smallskip
}

\tcr{ \(
\small
\begin{array}{ll}
r_3: & (color \ high) \land (strength \ high)  \\ 
&w(r_3) = 0.65, \ AD(r_3(x),x) = 0.30 
\end{array}
\)
\smallskip
}

\tcr{ \(
\small
\begin{array}{ll}
r_4: & (color \ high) \land (strength \ very \ high)  \\ 
&w(r_4) = 0.12, \ AD(r_4(x),x) = 0.06 
\end{array}
\)
\smallskip
}

\item \tcr Class $Stout$
\item \tcr Class $Barleywine$
\tcr{ \(
\small
\begin{array}{ll}
r_5: & (color \ low) \land (strength \ high)  \\ 
&w(r_5) = 0.07, \ AD(r_5(x),x) = 0.03 
\end{array}
\)
\smallskip
}

\tcr{ \(
\small
\begin{array}{ll}
r_6: & (color \ low) \land (stren

In [ ]:
idx = 1
rs = [r for r in i_cf if r[1] > 0][:4]
for r in rs:
    print(latex_rule(r, fuzzy_element, idx))
    idx += 1

In [123]:
def latex_cf(cf_rule, idx):
    rule, d = cf_rule
    lr = ""
    lr += f"r_c_{idx}: &({rule.antecedent[0][0]} \ {VARIABLES_DICT[rule.antecedent[0][0]][rule.antecedent[0][1]]}) \land ({rule.antecedent[1][0]} \ {VARIABLES_DICT[rule.antecedent[1][0]][rule.antecedent[1][1]]})  \\rightarrow {rule.consequent}\\\\ \n"
    lr += f"&d(r_c_{idx}, x) = {d:.2f}\\\\"
    return lr

In [124]:
idx = 1
rs = [r for r in i_cf if r[1] > 0][:4]
for r in rs:
    print(latex_cf(r, idx))
    idx += 1

r_c_1: &(color \ high) \land (strength \ very \ high)  \rightarrow Barleywine\\ 
&d(r_c_1, x) = 0.06\\
r_c_2: &(color \ low) \land (strength \ high)  \rightarrow Barleywine\\ 
&d(r_c_2, x) = 0.11\\
r_c_3: &(color \ very \ low) \land (bitterness \ very \ high)  \rightarrow Barleywine\\ 
&d(r_c_3, x) = 0.11\\
r_c_4: &(color \ high) \land (strength \ high)  \rightarrow Barleywine\\ 
&d(r_c_4, x) = 0.16\\


In [125]:
idx = 1
rs = [r for r in f_cf if r[1] > 0][:4]
for r in rs:
    print(latex_cf(r, idx))
    idx += 1

r_c_1: &(color \ low) \land (strength \ high)  \rightarrow Barleywine\\ 
&d(r_c_1, x) = 0.08\\
r_c_2: &(color \ high) \land (strength \ high)  \rightarrow Barleywine\\ 
&d(r_c_2, x) = 0.10\\
r_c_3: &(color \ low) \land (strength \ very \ high)  \rightarrow Barleywine\\ 
&d(r_c_3, x) = 0.12\\
r_c_4: &(color \ high) \land (strength \ very \ high)  \rightarrow Barleywine\\ 
&d(r_c_4, x) = 0.15\\
